## DATASET CREATION

Unfortunately, the dataset I will work with does not come as one already done dataset - as it is mostly usual in Data Science projects. <br /> 
To get the sheer experience of how a normal Data Science job looks like I also wanted to dive into this issue. By experience I can tell that this takes a shit ton of time - like almost 80% - of preparing the data. <br /> 
Since the data usually shares the same countries and lists mostly the same years of the recording I used this as a primary key where I connect the datapoints with each other. <br /> 
However, the  countries are listed in rows along with the year of the recording - I want to have a final dataset that looks like follows: <br /> 

|Country | Afghanistan | Albania | ... | Zimbabwe | 
| ----- | ----------  | ------ | ----- | ------- | 
|Alcohol consumption [l] | 0.2 | 2.4 | ... | 0.01 |
|Human Develpment Index (HDI) | 0.1 | 0.15 | ... | 0.1 | 
|... | ... | ... | ... | ... |
|Healthcare Expenditure [$] | 13.322 | 15.211 | ... | 1.039 |

Thus, I have to transpose each of the countries and record each of the years as seperate entry in the dataset. <br /> 

All the data is [publicly available](https://ourworldindata.org), and this source is trusted by many notorious companies such as Vox, The Ney York times and even the top universities of this world like MIT, Oxford, Stanford. <br /> 
Hence, I assume that this data is rather based on actual recording from the respective country. <br /> 
Even the United Nation published their records in this page and I bet that these folks do some amazing work, which we can trust. <br />


But now let's not waste too much with the explaination part and go straight into how I merged the datafiles to one huge on. <br /> 

In [1]:
import os 
import sys
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
datapath = os.path.join(os.path.join(os.getcwd(), 'data'), 'Health')
datapath

'C:\\Users\\Lenny\\Documents\\Studium_Robotics (M.Sc.)\\03_Semester 3 - Oslo ERASMUS\\01_Applied Data Analysis and Machine Learning\\Project 3'

In [4]:
dataFileNames = [f for f in os.listdir(datapath) if os.path.isfile(os.path.join(datapath, f))]
type4Cols = []
type7Cols = []
manualLists = []
max_countries = 0
countries_list = []

# the datasets mostly have the same size of 4 columns and same setup so let's get those first

for file in dataFileNames: 
    try:
        df = pd.read_csv(os.path.join(datapath, str(file) ) , encoding='latin-1')
    except: 
        print(file)
        manualLists.append(file)
    if df.shape[1] == 4: # one type of files (4 columns) 
        type4Cols.append(file) 
        countries = list(set(df["Entity"]))
        if len(countries) > max_countries: 
            max_countries = len(countries)
            countries_list = countries
    elif df.shape[1] == 7: 
        type7Cols.append(file) 
    else: # manual shit  to do then 
        manualLists.append(file)
print( max_countries, countries_list)

C:\Users\Lenny\AppData\Local\Programs\Python\Python36\lib\site-packages\IPython\core\interactiveshell.py:3020: DtypeWarning: Columns (0) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


262 ['Liechtenstein', 'South Africa', 'Somalia', 'Bosnia and Herzegovina', 'Bangladesh', 'Hong Kong', 'Swaziland', 'Arab World', 'Barbados', 'Dominica', 'Maldives', 'United States Virgin Islands', 'Venezuela', 'Lithuania', 'Hungary', 'France', 'Mozambique', 'Democratic Republic of Congo', 'British Virgin Islands', 'Belarus', 'Russia', 'High income', 'Peru', 'Guatemala', 'Caribbean small states', 'Ghana', 'Egypt', 'Kenya', 'Other small states', 'Serbia', 'Canada', 'Guinea', 'Central African Republic', 'Algeria', 'Italy', 'Lesotho', 'North America', 'Colombia', 'Gibraltar', 'Haiti', 'Ireland', 'Papua New Guinea', 'Rwanda', 'European Union', 'Morocco', 'Togo', 'Montenegro', 'Gambia', 'Netherlands', 'Solomon Islands', 'Nauru', 'Jordan', 'Low & middle income', 'Moldova', 'South Korea', 'Libya', 'American Samoa', 'Costa Rica', 'Luxembourg', 'Isle of Man', 'Congo', 'Tajikistan', 'Burundi', 'Cape Verde', 'IBRD only', 'Indonesia', 'Portugal', 'Saudi Arabia', 'Vietnam', 'Channel Islands', 'North

In [5]:
df.head(10)

,Entity,Code,Year,Wine Consumption [l]
0,Afghanistan,AFG,1961,0.0
1,Afghanistan,AFG,1962,0.0
2,Afghanistan,AFG,1963,0.0
3,Afghanistan,AFG,1964,0.0
4,Afghanistan,AFG,1965,0.0
5,Afghanistan,AFG,1966,0.0
6,Afghanistan,AFG,1967,0.0
7,Afghanistan,AFG,1968,0.0
8,Afghanistan,AFG,1969,0.0
9,Afghanistan,AFG,1970,0.0


In [6]:
# create dataframe where we want to paste everything inside
final_df = pd.DataFrame(data = np.zeros( (10000000, max_countries) ),
                        index = np.arange(10000000),
                        columns = [ country for country in countries_list])
# save indices as strings to get meaningful names
final_df.index = final_df.index.map(str)
final_df.shape

(10000000, 262)

In [13]:
# testing to get values of the specific entries and Co
columns = list(df.columns)
b = df[df['Entity'] == 'France']
c = b[b['Year'] == 1999]#[columns[-1]]
d = b[b['Year'] == 1999][columns[-1]].sum()
print(d)
c
#df.groupby(["Entity", 'Year']).describe()

7.8


,Entity,Code,Year,Wine Consumption [l]
2840,France,FRA,1999,7.8


In [36]:
%%time
# paste the values into the final_df from each single list
nextListIndex = 0

try: 
    for lists in type4Cols:
        ## Read the file and get the all countries along with their reported years
        print("{} list out of {}, Index: {}, Name: {}".format(type4Cols.index(lists), len(type4Cols), nextListIndex, lists))
        df = pd.read_csv(os.path.join(datapath, str(lists) ) , encoding='latin-1')
        
        # get the col names, unique countries and unique years
        columns = list(df.columns)
        countries = list(set(df["Entity"]))
        years = list(set(df['Year']))
        
        # get a list of all the index/row names
        indexNamesArr = final_df.index.values

        ## groupby countries and then years accordingly
        #df.groupby(["Entity", 'Year'])
        
        ## BREAK TO CHECK THE CORRECT VALUES AFTER 3 LISTS
        if type4Cols.index(lists) == 3: 
            break

        firstListRun = False # flag for renaming the indices

        # loop thru every country in the list
        for country in countries:
            # take a dataframe for one country at a time
            country_df = df[df['Entity'] == country]

            # loop thru every year within that country - assuming the years are in the same order for every country
            for year in years:
                
                # rename the indices only if it is the very first run for the country
                if not firstListRun:
                    indexName = str(columns[-1]) + ' in ' + str(year)
                    indexNamesArr[nextListIndex] = indexName
                    nextListIndex += 1
                    if nextListIndex % 20 == 0: 
                        print(indexName)

                ## get the proper value and fill empty ones, if not available, fill it with the average
                # note: .sum() is only having one element anyway, just done to get the value as a float not an array
                value = country_df[country_df["Year"] == year][columns[-1]].sum() if not country_df[country_df["Year"] == year][columns[-1]].empty else country_df[columns[-1]].mean()
                # get the name of the row
                idxName = str(columns[-1]) + ' in ' + str(year)
                # assign the value in the merged df with the value 
                final_df.iat[list(final_df.index.values).index(idxName), countries.index(country)] = value # .iat[row, col]

            ## get the proper index for the next list to begin with   
            firstListRun = True
            # print progress
            if countries.index(country) % 15 == 0:
                print("{} country out of {}".format(countries.index(country), len(countries)))
            
except Exception as e: 
    print("Next List Index in line: {} of {}, Matrix Size: {}, list: {}".format(i, nextListIndex, final_df.shape[0], lists))
    print(e)
    sys.exit()

0 list out of 60, Index: 0, Name: agricultural-area-per-capita.csv
Agriculutral are per capita [h/person] in 1980
Agriculutral are per capita [h/person] in 2000
0 country out of 187
1 country out of 187
2 country out of 187
3 country out of 187
4 country out of 187
5 country out of 187
6 country out of 187
7 country out of 187
8 country out of 187
9 country out of 187
10 country out of 187
11 country out of 187
12 country out of 187
13 country out of 187
14 country out of 187
15 country out of 187
16 country out of 187
17 country out of 187
18 country out of 187
19 country out of 187
20 country out of 187
21 country out of 187
22 country out of 187
23 country out of 187
24 country out of 187
25 country out of 187
26 country out of 187
27 country out of 187
28 country out of 187
29 country out of 187
30 country out of 187
31 country out of 187
32 country out of 187
33 country out of 187
34 country out of 187
35 country out of 187
36 country out of 187
37 country out of 187
38 country ou

170 country out of 190
171 country out of 190
172 country out of 190
173 country out of 190
174 country out of 190
175 country out of 190
176 country out of 190
177 country out of 190
178 country out of 190
179 country out of 190
180 country out of 190
181 country out of 190
182 country out of 190
183 country out of 190
184 country out of 190
185 country out of 190
186 country out of 190
187 country out of 190
188 country out of 190
189 country out of 190
2 list out of 60, Index: 54, Name: annual-healthcare-expenditure-per-capita.csv
annual healthcare spending [$] in 2000
0 country out of 236
1 country out of 236
2 country out of 236
3 country out of 236
4 country out of 236
5 country out of 236
6 country out of 236
7 country out of 236
8 country out of 236
9 country out of 236
10 country out of 236
11 country out of 236
12 country out of 236
13 country out of 236
14 country out of 236
15 country out of 236
16 country out of 236
17 country out of 236
18 country out of 236
19 country ou

In [38]:
final_df[50:60]

,Liechtenstein,South Africa,Somalia,Bosnia and Herzegovina,Bangladesh,Hong Kong,Swaziland,Arab World,Barbados,Dominica,...,Europe & Central Asia,Myanmar,Benin,Bulgaria,Latin America & Caribbean (IDA & IBRD),Jamaica,Greece,Eritrea,Bahamas,Namibia
Agriculutral are per capita [h/person] in 2011,1.855166,0.560302,0.059714,0.000775,1.008251,0.049645,0.352113,0.023795,0.732179,0.921781,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Agriculutral are per capita [h/person] in 2012,1.848605,0.562337,0.058955,0.000713,0.992689,0.049470,0.347222,0.023373,0.721082,0.938639,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Agriculutral are per capita [h/person] in 2013,1.834944,0.563594,0.058163,0.000708,0.977600,0.049123,0.347222,0.022899,0.710409,0.958369,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
Alcohol-attributable deaths [%] in 2012,6.400000,0.300000,4.100000,0.600000,3.300000,3.300000,3.700000,1.900000,10.800000,30.900000,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
annual healthcare spending [$] in 1995,554.926813,112.553535,33.847622,182.376821,291.042096,475.105476,304.923564,154.965483,605.996771,332.148748,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
annual healthcare spending [$] in 1996,591.541768,229.659488,32.440083,205.592236,302.563796,477.025876,320.671077,178.045521,525.840775,371.166402,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
annual healthcare spending [$] in 1997,615.471244,263.671216,25.629278,199.691359,324.799812,518.453110,339.728887,196.756772,478.693863,418.439612,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
annual healthcare spending [$] in 1998,608.318584,308.200467,26.706320,218.069979,356.426797,519.199357,347.710987,252.683377,521.095548,485.071953,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
annual healthcare spending [$] in 1999,624.670572,364.137124,28.324157,216.533987,373.424780,553.604329,359.752491,264.876561,540.245674,495.522360,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
annual healthcare spending [$] in 2000,622.172258,314.572710,30.351706,218.972352,382.665594,593.852786,299.928136,301.929014,560.703139,558.238536,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [32]:
indexNames = final_df.index.values
idxName = 'Alcohol-attributable deaths [%] in 2012'
index = list(final_df.index.values).index(idxName)
#indexNames[50:65]
index

53

In [ ]:
final_df.loc[ 'b' : 'd'] #index it properly

In [ ]:
final_df.to_csv(os.path.join(datapath, "Merged UN Data.csv"))

BELOW HERE IS JUST THE PLAYGROUND TO TRY SOME STUFF OUT -> The dataset should be done in the top part

In [ ]:
df.groupby('Entity')['Wine Consumption'].sum().sort_values().tail(5)

In [ ]:
df[df['Entity'] == 'Germany']['Wine Consumption'].plot.hist(bins=20)